In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle
import os

import psutil
from sklearn.preprocessing import OneHotEncoder


In [8]:
def variables_por_tamaño():
    variables = {k: sys.getsizeof(v) for k, v in globals().items() if not k.startswith('_')}
    
    variables_en_gb = {k: v / 1024**3 for k, v in variables.items()}
    
    tipos_variables = {k: type(v).__name__ for k, v in globals().items() if not k.startswith('_')}
    
    df = pd.DataFrame({'Variable': list(variables_en_gb.keys()),
                       'Tipo': list(tipos_variables.values()),
                       'Tamaño_GB': list(variables_en_gb.values())})
    
    df = df.sort_values('Tamaño_GB', ascending=False).reset_index(drop=True)
    
    # Mostrar la memoria disponible
    memoria_disponible()
    
    return df



def Gb_por_variable(variable):
    
    print(sys.getsizeof(variable)/(1024**3))
    memoria_disponible()

def memoria_disponible():
    mem = psutil.virtual_memory()
    mem_total_gb = mem.total / (1024 ** 3)  # Memoria total en GB
    mem_disponible_gb = mem.available / (1024 ** 3)  # Memoria disponible en GB

    print("Memoria Total:", mem_total_gb, "GB")
    print("Memoria Disponible:", mem_disponible_gb, "GB")


In [32]:
archivo= 'Pkl/df_temp.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    submatrices_diccionario_trabajadas = pickle.load(archivo_pkl)

In [4]:
archivo= 'Pkl/train_dict_Columnas_worked.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    train_dict_Columnas_worked = pickle.load(archivo_pkl)
train_dict={}
train_dict['Columnas_Worked']=train_dict_Columnas_worked
memoria_disponible()

Memoria Total: 15.411502838134766 GB
Memoria Disponible: 2.982524871826172 GB


In [42]:
archivo= 'Pkl/Cluster_Info.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    Cluster_Info = pickle.load(archivo_pkl)
memoria_disponible()

Memoria Total: 15.411502838134766 GB
Memoria Disponible: 2.6760482788085938 GB


In [16]:
def explorar_una_matriz(submatriz):
    return pd.DataFrame(submatrices_diccionario_trabajadas[submatriz],columns=train_dict['Columnas_Worked'])

col_dummies=  ['Semana']
indices_cat = [i for i, col in enumerate(train_dict['Columnas_Worked']) if col in col_dummies]
indices_num = [i for i, col in enumerate(train_dict['Columnas_Worked']) if col not in col_dummies]

Dict_Columnas_Dummizadas = {}
Dict_nombres_de_columnas_df_dummizadas = {}

def Duminizar_numpy(submatriz):
    matriz_final=submatrices_diccionario_trabajadas[submatriz][:,indices_num]
    columnas_final = train_dict['Columnas_Worked'][indices_num]
    
    for col in indices_cat:
        onehot_encoder = OneHotEncoder(sparse_output=False)
        one_hot_column = onehot_encoder.fit_transform(submatrices_diccionario_trabajadas[submatriz][:,col].reshape(-1, 1))
        one_hot_column_names=  onehot_encoder.get_feature_names_out([train_dict['Columnas_Worked'][col]])
        columnas_final = np.concatenate((columnas_final ,one_hot_column_names))
        matriz_final = np.concatenate((matriz_final,one_hot_column), axis=1)

    Dict_Columnas_Dummizadas[submatriz] = matriz_final
    Dict_nombres_de_columnas_df_dummizadas[submatriz] = columnas_final
    submatrices_diccionario_trabajadas[submatriz]= 0
    del matriz_final,columnas_final 

valores_unicos = np.unique([valor for array in Dict_nombres_de_columnas_df_dummizadas.values() for valor in array])
Dict_all_Columns = {}

def ajustar_a_columnas_restantes(submatriz):
    df_temp = pd.DataFrame(Dict_Columnas_Dummizadas[submatriz],columns=Dict_nombres_de_columnas_df_dummizadas[submatriz])
    columnas_faltantes = [col for col in valores_unicos if col not in df_temp.columns]

    for col in columnas_faltantes:
        df_temp[col] = 0  # Puedes cambiar el valor si deseas algo diferente

    return df_temp[valores_unicos].to_numpy()

def unir_y_borrar():
    for submatriz in Dict_all_Columns.keys():
        print(str(submatriz)+ '/' + str(len(list(Dict_all_Columns.keys()))) +'|'+str(round(submatriz/len(list(Dict_all_Columns.keys()))*100,3))+'%')
        if submatriz ==1:
            All_junto = Dict_all_Columns[submatriz]
        else:
            All_junto = np.concatenate([All_junto,Dict_all_Columns[submatriz]],axis=0)
        Dict_all_Columns[submatriz]=0
    return All_junto
  

In [27]:
#PRoceso completo
archivo= 'Pkl/df_temp.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    submatrices_diccionario_trabajadas = pickle.load(archivo_pkl)

archivo= 'Pkl/train_dict_Columnas_worked.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    train_dict_Columnas_worked = pickle.load(archivo_pkl)
train_dict={}
train_dict['Columnas_Worked']=train_dict_Columnas_worked
memoria_disponible()



for submatriz in submatrices_diccionario_trabajadas.keys():
    print(str(submatriz)+ '/' + str(len(list(submatrices_diccionario_trabajadas.keys()))) +'|'+str(round(submatriz/len(list(submatrices_diccionario_trabajadas.keys()))*100,3))+'%')
    Duminizar_numpy(submatriz)

Dict_Columnas_Dummizadas
valores_unicos = np.unique([valor for array in Dict_nombres_de_columnas_df_dummizadas.values() for valor in array])

for submatriz in Dict_Columnas_Dummizadas.keys():
    print(str(submatriz)+ '/' + str(len(list(Dict_nombres_de_columnas_df_dummizadas.keys()))) +'|'+str(round(submatriz/len(list(Dict_nombres_de_columnas_df_dummizadas.keys()))*100,3))+'%')
    Dict_all_Columns[submatriz]=ajustar_a_columnas_restantes(submatriz)
    Dict_nombres_de_columnas_df_dummizadas[submatriz]=0

Matriz_final = unir_y_borrar()

Memoria Total: 15.411502838134766 GB
Memoria Disponible: 0.5375099182128906 GB
1/149|0.671%
2/149|1.342%
3/149|2.013%
4/149|2.685%
5/149|3.356%
6/149|4.027%
7/149|4.698%
8/149|5.369%
9/149|6.04%
10/149|6.711%
11/149|7.383%
12/149|8.054%
13/149|8.725%
14/149|9.396%
15/149|10.067%
16/149|10.738%
17/149|11.409%
18/149|12.081%
19/149|12.752%
20/149|13.423%
21/149|14.094%
22/149|14.765%
23/149|15.436%
24/149|16.107%
25/149|16.779%
26/149|17.45%
27/149|18.121%
28/149|18.792%
29/149|19.463%
30/149|20.134%
31/149|20.805%
32/149|21.477%
33/149|22.148%
34/149|22.819%
35/149|23.49%
36/149|24.161%
37/149|24.832%
38/149|25.503%
39/149|26.174%
40/149|26.846%
41/149|27.517%
42/149|28.188%
43/149|28.859%
44/149|29.53%
45/149|30.201%
46/149|30.872%
47/149|31.544%
48/149|32.215%
49/149|32.886%
50/149|33.557%
51/149|34.228%
52/149|34.899%
53/149|35.57%
54/149|36.242%
55/149|36.913%
56/149|37.584%
57/149|38.255%
58/149|38.926%
59/149|39.597%
60/149|40.268%
61/149|40.94%
62/149|41.611%
63/149|42.282%
64/14

In [32]:
nombre_archivo = 'Pkl/df_final.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(Matriz_final, archivo_pkl)

In [35]:
valores_unicos

array(['Canal_ID', 'Cluster_Agencia_ID', 'Cluster_Cliente_ID',
       'Cluster_Producto', 'Cluster_Ruta_SAK', 'Demanda_uni_equil',
       'Dev_proxima', 'Dev_uni_proxima', 'Precio_Promedio',
       'Ratio Devoluciones_unit', 'Ratio_Devoluciones_valor',
       'Semana_3.0', 'Semana_4.0', 'Semana_5.0', 'Semana_6.0',
       'Semana_7.0', 'Semana_8.0', 'Semana_9.0', 'Venta_hoy',
       'Venta_uni_hoy'], dtype='<U24')

In [33]:
Matriz_final_ventas = Matriz_final[:,]

array([[ 7.  ,  6.  ,  0.  , ...,  0.  , 25.14,  3.  ],
       [ 7.  ,  6.  ,  0.  , ...,  0.  , 33.52,  4.  ],
       [ 7.  ,  6.  ,  0.  , ...,  0.  , 39.32,  4.  ],
       ...,
       [ 1.  ,  7.  ,  0.  , ...,  1.  , 51.  ,  4.  ],
       [ 1.  ,  7.  ,  0.  , ...,  1.  , 26.94,  3.  ],
       [ 1.  ,  7.  ,  0.  , ...,  1.  , 10.4 ,  1.  ]])